Hier werden die Portfolio-Methoden angewandt und mithilfe der Performance-Metriken bewertet. Die Ergebnisse werden in CSV-Dateien festgehalten. Das Akquirieren, modifizieren und beschreiben der Daten befindet sich der Übersicht halber im Notebook Datenbasis. Die Daten werden dort als CSV exportiert und hier als Dataframe wieder eingelesen.

Die Korrektheit der implementierten Methoden und Metriken wird in dem separaten Notebook Verifizierung anhand eines kleinen Teilbestandes aufgezeigt, siehe dazu auch beigefügte Excel-Dateien, in denen die Portfolio-Gewichte "per Hand" nachgerechnet und mit den Ergebnissen aus der Python-Implementierung abgeglichen werden.

ACHTUNG: LANGE LAUFZEIT!

## Importe

In [1]:
#Import der notwendigen Bibliotheken, Installation mit pip bereits erfolgt

import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

## Einlesen der Daten

In [2]:
#Bearbeitete JSE-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'df_JSE_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [3]:
df_JSE.info

<bound method DataFrame.info of             4SI.JO   ABG.JO  ACL.JO  ACT.JO       ADH.JO  AEL.JO  \
Date                                                               
2005-06-01     NaN   8240.0  4980.0   130.0   133.701508     NaN   
2005-06-02     NaN   8250.0  4949.0   130.0   131.749695     NaN   
2005-06-03     NaN   8261.0  4949.0   130.0   131.749695     NaN   
2005-06-06     NaN   8290.0  5023.0   130.0   132.725601     NaN   
2005-06-07     NaN   8345.0  4845.0   149.0   131.749695     NaN   
...            ...      ...     ...     ...          ...     ...   
2025-05-26    70.0  16845.0   109.0   160.0  3199.000000  2350.0   
2025-05-27    70.0  16848.0   105.0   160.0  3210.000000  2350.0   
2025-05-28    73.0  17300.0   101.0   174.0  3283.000000  2355.0   
2025-05-29    70.0  17579.0   106.0   166.0  3247.000000  2362.0   
2025-05-30    70.0  17332.0   116.0   160.0  3215.000000  2358.0   

                  AFE.JO  AFH.JO  AFT.JO        AGL.JO  ...  VKE.JO   VOD.JO  \
Dat

In [4]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_SP500 = pd.read_csv(
    'df_SP500_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [5]:
df_SP500.info

<bound method DataFrame.info of                      A  AAL  AAP        AAPL        ABBV        ABNB  \
Date                                                                   
2005-06-01   17.296137  NaN  NaN    1.439286         NaN         NaN   
2005-06-02   17.303289  NaN  NaN    1.430000         NaN         NaN   
2005-06-03   17.210300  NaN  NaN    1.365714         NaN         NaN   
2005-06-06   17.539343  NaN  NaN    1.354286         NaN         NaN   
2005-06-07   17.324751  NaN  NaN    1.305000         NaN         NaN   
...                ...  ...  ...         ...         ...         ...   
2025-05-23  108.529999  NaN  NaN  195.270004  183.259995  126.720001   
2025-05-27  111.260002  NaN  NaN  200.210007  185.720001  129.399994   
2025-05-28  110.879997  NaN  NaN  200.419998  183.089996  128.669998   
2025-05-29  113.279999  NaN  NaN  199.949997  185.619995  128.360001   
2025-05-30  111.919998  NaN  NaN  200.850006  186.110001  129.000000   

                   ABT       AC

In [6]:
#Reduzierter SP500-Bestand ohne ausgeschiedene Unternehmen und auf Größe des JSE runtergebrochen

df_SP500_reduziert = pd.read_csv(
    'sp500_gefiltert_neuneu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    dayfirst=True
)

In [7]:
df_SP500_reduziert.info

<bound method DataFrame.info of                      A        AAPL         ABT         ACN         ADI  \
Date                                                                     
2005-06-01   17.296137    1.439286         NaN         NaN         NaN   
2005-06-02   17.303289    1.430000         NaN         NaN         NaN   
2005-06-03   17.210300    1.365714         NaN         NaN         NaN   
2005-06-06   17.539343    1.354286         NaN         NaN         NaN   
2005-06-07   17.324751    1.305000         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2025-05-23  108.529999  195.270004  131.300003  309.579987  210.470001   
2025-05-27  111.260002  200.210007  132.940002  315.429993  216.889999   
2025-05-28  110.879997  200.419998  132.020004  315.989990  215.619995   
2025-05-29  113.279999  199.949997  132.850006  317.730011  215.750000   
2025-05-30  111.919998  200.850006  133.580002  316.820007  213.979996   

     

## Definition der Methoden und Metriken

#### Equally-Weighted Portfolio

In [8]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

#### Minimum-Variance Portfolio

In [9]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

#### Maximum Diversification Portfolio

In [10]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [11]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=matrix, method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

#### Inverse Volatility Portfolio

In [12]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

#### Equal Risk Contribution Portfolio

In [13]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [14]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

#### Hierarchical Risk Parity

In [15]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [16]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [17]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [18]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [19]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [20]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

#### Hierarchical Equal Risk Contribution Portfolio

In [21]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

In [22]:
def herc_ledoit (returns):

    #Building the portfolio object
    port = hc(returns=returns)
    
    shrinkage_herc = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix_herc = pd.DataFrame(
        shrinkage_herc.covariance_,
        index = returns.columns,
        columns = returns.columns)

    #Estimate optimal portfolio:

    model='HERC'
    #correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          custom_cov = shrinkage_matrix_herc,
                          codependence='custom_cov',
                          rm=rm,
                          rf=rf,
                          method_cov='custom_cov',   #gemäß https://riskfolio-lib.readthedocs.io/en/latest/hcportfolio.html
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

#### Vergleichs-Metriken

In [23]:
def gesamt_return(gewichte, returns):

    daily_returns = returns @ gewichte
    total = np.prod(1 + daily_returns) - 1
    
    return total

In [24]:
def portfolio_return (gewichte, returns):
    
    avg_returns = []
    avg_returns = returns.mean()
    
    pf_return = sum([gewicht * rendite for gewicht, rendite in zip(gewichte, avg_returns)])
    
    return pf_return

In [25]:
def portfolio_variance (gewichte, returns):
    daily_returns = []
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    pf_var = np.var(daily_returns)
    
    return pf_var

In [26]:
def portfolio_volatility (gewichte, returns):
    variance = portfolio_variance(gewichte, returns)
    volatility = np.sqrt(variance)
    
    return volatility

In [27]:
def annualized_return(gewichte, returns):
    daily_returns = returns @ gewichte
    annual = (np.prod(1 + daily_returns))**(252/daily_returns.shape[0]) - 1
    
    return annual

In [28]:
def annualized_portfolio_return(gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    
    return pf_return*252

In [29]:
def annualized_portfolio_volatility(gewichte, returns):
    pf_volatility = portfolio_volatility(gewichte, returns)
    
    return pf_volatility*np.sqrt(252)

In [30]:
def sharpe_ratio (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [31]:
def sharpe_ratio_one (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [32]:
def sharpe_ratio_two (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [33]:
def annualized_sharpe_ratio (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [34]:
def annualized_sharpe_ratio_one (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [35]:
def annualized_sharpe_ratio_two (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [36]:
def sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [37]:
def sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [38]:
def sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [39]:
def annualized_sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [40]:
def annualized_sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [41]:
def annualized_sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [42]:
def maximum_drawdown (gewichte, returns):
    daily_returns = returns @ gewichte
    returns_kumuliert = (1 + daily_returns).cumprod()
    hochpunkte = returns_kumuliert.cummax()
    
    drawdowns = (returns_kumuliert - hochpunkte) / hochpunkte
    max_drawdown = drawdowns.min()
    
    #print(drawdowns)
    return max_drawdown

In [43]:
def calmar_ratio (gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    max_drawdown = maximum_drawdown(gewichte, returns)
    
    calmar_ratio = pf_return / (-max_drawdown)
    
    return calmar_ratio

In [44]:
def value_at_risk_func(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = np.percentile(sorted_daily_returns, 100*0.05)
    return value_at_risk

In [45]:
#https://github.com/malctaylor15/VaR-and-Expected-Shortfall-Python/blob/master/Expected%20Shortfall%20and%20Value%20at%20Risk.ipynb

def expected_shortfall(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = value_at_risk_func(gewichte, returns)
    
    expected_shortfall = daily_returns[daily_returns <= value_at_risk].mean()
    
    return expected_shortfall

#### Sammel-Funktionen zum Berechnen und Speichern der Metriken

In [46]:
def metriken_equally_weighted(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile1 = {"Methode": "Equally-Weighted",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile1[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile1])], ignore_index=True)

In [47]:
def metriken_minimum_variance(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile2 = {"Methode": "Minimum_Variance",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile2[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile2])], ignore_index=True)

In [48]:
def metriken_max_diversification(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile3 = {"Methode": "Max-Diversification",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile3[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile3])], ignore_index=True)

In [49]:
def metriken_inverse_volatility(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile4 = {"Methode": "Inverse-Volatility",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile4[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile4])], ignore_index=True)

In [50]:
def metriken_risk_parity(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile5 = {"Methode": "Risk-Parity",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile5[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile5])], ignore_index=True)

In [51]:
def metriken_hrp(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile6 = {"Methode": "HRP",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile6[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile6])], ignore_index=True)

In [52]:
def metriken_herc(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile7 = {"Methode": "HERC",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile7[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile7])], ignore_index=True)

## Erzeugung Ergebnisse S&P500

In [53]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [54]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    gewichte_equally_weighted = equally_weighted(training_daten)
    gewichte_min_variance= min_variance_opt(cov_matrix)
    gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": gewichte_equally_weighted,
    "Minimum-Variance": gewichte_min_variance,
    "Maximum-Diversification": gewichte_max_diversification,
    "Inverse-Volatility": gewichte_inverse_volatility,
    "Equal-Risk-Contribution": gewichte_risk_parity,
    "HRP": gewichte_hrp,
    "HERC": gewichte_herc
}
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df = pd.concat(portfolio_returns_list)
portfolio_returns_df = portfolio_returns_df.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.83622822322847e-05
            Iterations: 326
            Function evaluations: 85412
            Gradient evaluations: 326
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8444369611493534
            Iterations: 70
            Function evaluations: 17357
            Gradient evaluations: 66
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.166190914201733e-09
            Iterations: 19
            Function evaluations: 4979
            Gradient evaluations: 19
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.912643376317497e-05
            Iterations: 322
            Function evaluations: 84364
            Gradient evaluations: 322
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.797

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.574751809295369
            Iterations: 45
            Function evaluations: 12464
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6038161830066482e-09
            Iterations: 39
            Function evaluations: 10764
            Gradient evaluations: 39
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.549984244524742e-05
            Iterations: 291
            Function evaluations: 80316
            Gradient evaluations: 291
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5501553762276297
            Iterations: 42
            Function evaluations: 11636
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.362678543712232e-10
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.2975844188171925e-10
            Iterations: 84
            Function evaluations: 23353
            Gradient evaluations: 84
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00015139486526506186
            Iterations: 147
            Function evaluations: 40866
            Gradient evaluations: 147
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9302777484788625
            Iterations: 30
            Function evaluations: 8368
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.6488537881787093e-10
            Iterations: 84
            Function evaluations: 23353
            Gradient evaluations: 84
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00013901077694280492
            Iterations: 148
            Function evaluations: 42772
            Gradient evaluations: 148
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8013330280287727
            Iterations: 32
            Function evaluations: 9266
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.724114532549846e-10
            Iterations: 92
            Function evaluations: 26588
            Gradient evaluations: 92
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000139152362661429
            Iterations: 153
            Function evaluations: 44523
            Gradient evaluations: 153
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.79

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2688977077365253
            Iterations: 35
            Function evaluations: 10883
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0291630518964678e-09
            Iterations: 42
            Function evaluations: 13020
            Gradient evaluations: 42
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.075775085435204e-05
            Iterations: 256
            Function evaluations: 79872
            Gradient evaluations: 256
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2465491702642617
            Iterations: 37
            Function evaluations: 11579
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.512483264582795e-10
     

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.06614807078087
            Iterations: 25
            Function evaluations: 9998
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.73835129013812e-10
            Iterations: 67
            Function evaluations: 26733
            Gradient evaluations: 67
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00010487186411525202
            Iterations: 201
            Function evaluations: 80601
            Gradient evaluations: 201
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.051470628122572
            Iterations: 33
            Function evaluations: 12864
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.675661927779794e-10
          

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0748260324304574e-09
            Iterations: 75
            Function evaluations: 31050
            Gradient evaluations: 75
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.847504626382485e-05
            Iterations: 213
            Function evaluations: 88609
            Gradient evaluations: 213
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.181881095114811
            Iterations: 27
            Function evaluations: 11257
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4670731479461649e-09
            Iterations: 65
            Function evaluations: 27041
            Gradient evaluations: 65
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.976

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.865627124106826e-05
            Iterations: 260
            Function evaluations: 112060
            Gradient evaluations: 260
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.530391094395828
            Iterations: 45
            Function evaluations: 19441
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2933639981368081e-09
            Iterations: 46
            Function evaluations: 19826
            Gradient evaluations: 46
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.8315761239956976e-05
            Iterations: 239
            Function evaluations: 103249
            Gradient evaluations: 239
Optimization terminated successfully    (Exit mode 0)
            Current function value: -

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.4098864112860974
            Iterations: 47
            Function evaluations: 19804
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2024040715367831e-09
            Iterations: 47
            Function evaluations: 20634
            Gradient evaluations: 47
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.0246785013084796e-05
            Iterations: 266
            Function evaluations: 116775
            Gradient evaluations: 266
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3876940747776634
            Iterations: 43
            Function evaluations: 18919
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3183130173073275e-09
  

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.410436704562e-09
            Iterations: 27
            Function evaluations: 12043
            Gradient evaluations: 27
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.380565067459523e-05
            Iterations: 311
            Function evaluations: 139328
            Gradient evaluations: 311
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.3714830308240895
            Iterations: 80
            Function evaluations: 34120
            Gradient evaluations: 76
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5419093987421145e-09
            Iterations: 27
            Function evaluations: 12097
            Gradient evaluations: 27
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.24323

In [55]:
#Exportieren nach CSV
portfolio_returns_df.to_csv("renditensp500_neu_2jahre_f.csv")

## Ergebnis-Erzeugung JSE

In [56]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [57]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": JSE_gewichte_equally_weighted,
    "Minimum-Variance": JSE_gewichte_min_variance,
    "Maximum-Diversification": JSE_gewichte_max_diversification,
    "Inverse-Volatility": JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": JSE_gewichte_risk_parity,
    "HRP": JSE_gewichte_hrp,
    "HERC": JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    #Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_JSE = portfolio_returns_df_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.214356014733246e-05
            Iterations: 103
            Function evaluations: 6901
            Gradient evaluations: 103
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.466136913955411
            Iterations: 29
            Function evaluations: 1973
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4760500306010479e-09
            Iterations: 43
            Function evaluations: 2882
            Gradient evaluations: 43
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.306641209101051e-05
            Iterations: 128
            Function evaluations: 8576
            Gradient evaluations: 128
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.51589

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6906209336261457
            Iterations: 37
            Function evaluations: 2662
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.927905920871688e-10
            Iterations: 43
            Function evaluations: 3053
            Gradient evaluations: 43
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.34988484876616e-05
            Iterations: 171
            Function evaluations: 12312
            Gradient evaluations: 171
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8661844915990486
            Iterations: 42
            Function evaluations: 3068
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.088908855309106e-10
          

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0690863377304144e-09
            Iterations: 62
            Function evaluations: 4650
            Gradient evaluations: 62
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.775272151517432e-05
            Iterations: 151
            Function evaluations: 11174
            Gradient evaluations: 151
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9023024554619066
            Iterations: 41
            Function evaluations: 3072
            Gradient evaluations: 41
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0006278735501991e-09
            Iterations: 62
            Function evaluations: 4588
            Gradient evaluations: 62
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.86389

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.17830837802119e-05
            Iterations: 146
            Function evaluations: 11388
            Gradient evaluations: 146
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1121926156912445
            Iterations: 42
            Function evaluations: 3313
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2331327522714815e-09
            Iterations: 73
            Function evaluations: 5695
            Gradient evaluations: 73
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.561125500293392e-05
            Iterations: 155
            Function evaluations: 12245
            Gradient evaluations: 155
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.170

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.516032274807825
            Iterations: 48
            Function evaluations: 4074
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.6379181358279445e-09
            Iterations: 45
            Function evaluations: 3781
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4778627618983793e-05
            Iterations: 221
            Function evaluations: 18785
            Gradient evaluations: 221
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.5942299710188763
            Iterations: 57
            Function evaluations: 4895
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.055256101589123e-09
        

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.762371108243836
            Iterations: 61
            Function evaluations: 8051
            Gradient evaluations: 61
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1283954782169105e-09
            Iterations: 69
            Function evaluations: 9039
            Gradient evaluations: 69
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.123843529887515e-05
            Iterations: 193
            Function evaluations: 25476
            Gradient evaluations: 193
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8110524655358544
            Iterations: 64
            Function evaluations: 8515
            Gradient evaluations: 64
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.684786603873544e-10
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.874145771911388e-10
            Iterations: 74
            Function evaluations: 10360
            Gradient evaluations: 74
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.7810529746683515e-05
            Iterations: 198
            Function evaluations: 27919
            Gradient evaluations: 198
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.994281234882012
            Iterations: 76
            Function evaluations: 10255
            Gradient evaluations: 72
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0987761788234156e-09
            Iterations: 64
            Function evaluations: 9025
            Gradient evaluations: 64
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.8119

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4545933314415494e-05
            Iterations: 202
            Function evaluations: 30503
            Gradient evaluations: 202
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.1065129791412325
            Iterations: 86
            Function evaluations: 13097
            Gradient evaluations: 86
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.85920519621273e-09
            Iterations: 49
            Function evaluations: 7400
            Gradient evaluations: 49
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.387386327306118e-05
            Iterations: 198
            Function evaluations: 29502
            Gradient evaluations: 198
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.11

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.2157020563895395
            Iterations: 81
            Function evaluations: 12502
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.214671931100365e-09
            Iterations: 45
            Function evaluations: 6885
            Gradient evaluations: 45
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.051439145644047e-05
            Iterations: 203
            Function evaluations: 31059
            Gradient evaluations: 203
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.248786998204767
            Iterations: 96
            Function evaluations: 14225
            Gradient evaluations: 92
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.113085030640999e-09
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.965516656713942e-09
            Iterations: 42
            Function evaluations: 6469
            Gradient evaluations: 42
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2620819333160845e-05
            Iterations: 249
            Function evaluations: 38346
            Gradient evaluations: 249
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.983367062935144
            Iterations: 98
            Function evaluations: 15244
            Gradient evaluations: 98
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.8696497448213444e-09
            Iterations: 41
            Function evaluations: 6315
            Gradient evaluations: 41
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.29644

In [58]:
#Exportieren nach CSV
portfolio_returns_df_JSE.to_csv("returnsJSE_neu_2jahre_f.csv")

## Ledoit-Wolf Shrinkage

In [59]:
#Definition von Ledoit-Shrinkage

def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    return shrinkage_matrix

### Auswertung S&P500 mit Ledoit Wolf

In [60]:
returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [61]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_gewichte_min_variance,
    "Maximum-Diversification": shrinked_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_gewichte_risk_parity,
    "HRP": shrinked_gewichte_hrp,
    "HERC": shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked = portfolio_returns_df_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7982987444435394e-05
            Iterations: 311
            Function evaluations: 81482
            Gradient evaluations: 311
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.890980944238778
            Iterations: 67
            Function evaluations: 17622
            Gradient evaluations: 67
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0583075101385325e-09
            Iterations: 19
            Function evaluations: 4978
            Gradient evaluations: 19
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.877486184836653e-05
            Iterations: 283
            Function evaluations: 74147
            Gradient evaluations: 283
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6006221492082235
            Iterations: 45
            Function evaluations: 12467
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5419714754805422e-09
            Iterations: 39
            Function evaluations: 10765
            Gradient evaluations: 39
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.579323391495113e-05
            Iterations: 296
            Function evaluations: 81696
            Gradient evaluations: 296
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.575129619364104
            Iterations: 45
            Function evaluations: 12467
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8495411032817505e-09
     

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7601287205014498e-10
            Iterations: 93
            Function evaluations: 25855
            Gradient evaluations: 93
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001510300855273692
            Iterations: 128
            Function evaluations: 35584
            Gradient evaluations: 128
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9526287514349117
            Iterations: 30
            Function evaluations: 8364
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.5340817073770546e-10
            Iterations: 84
            Function evaluations: 23353
            Gradient evaluations: 84
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00013872264930836893
            Iterations: 125
            Function evaluations: 36125
            Gradient evaluations: 125
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.83125808219701
            Iterations: 28
            Function evaluations: 8117
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.638892257744304e-10
            Iterations: 92
            Function evaluations: 26588
            Gradient evaluations: 92
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00013871902507060123
            Iterations: 141
            Function evaluations: 41031
            Gradient evaluations: 141
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.82

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2879184115003732
            Iterations: 36
            Function evaluations: 11195
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0043894274516348e-09
            Iterations: 44
            Function evaluations: 13640
            Gradient evaluations: 44
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0697082316336616e-05
            Iterations: 256
            Function evaluations: 79872
            Gradient evaluations: 256
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2647072419672902
            Iterations: 34
            Function evaluations: 10642
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.314858924370984e-10
    

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1103504758646277
            Iterations: 27
            Function evaluations: 10796
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.303690905440099e-10
            Iterations: 67
            Function evaluations: 26733
            Gradient evaluations: 67
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00010286913610165605
            Iterations: 195
            Function evaluations: 78195
            Gradient evaluations: 195
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0968851066212015
            Iterations: 26
            Function evaluations: 10451
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.247347534897057e-10
     

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3533080634525048e-09
            Iterations: 65
            Function evaluations: 26910
            Gradient evaluations: 65
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.697005804508942e-05
            Iterations: 202
            Function evaluations: 84032
            Gradient evaluations: 202
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.222903242376857
            Iterations: 30
            Function evaluations: 12508
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0575277716356905e-09
            Iterations: 76
            Function evaluations: 31616
            Gradient evaluations: 76
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.814

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.783711962510311e-05
            Iterations: 273
            Function evaluations: 117663
            Gradient evaluations: 273
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5588235150664276
            Iterations: 68
            Function evaluations: 27651
            Gradient evaluations: 64
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2479109817840664e-09
            Iterations: 46
            Function evaluations: 19826
            Gradient evaluations: 46
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.770973199399202e-05
            Iterations: 240
            Function evaluations: 103680
            Gradient evaluations: 240
Optimization terminated successfully    (Exit mode 0)
            Current function value: -

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.4293907481955266
            Iterations: 38
            Function evaluations: 16720
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1747739502329904e-09
            Iterations: 47
            Function evaluations: 20634
            Gradient evaluations: 47
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.958732141509823e-05
            Iterations: 240
            Function evaluations: 105361
            Gradient evaluations: 240
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.4078808120912614
            Iterations: 45
            Function evaluations: 19359
            Gradient evaluations: 44
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.289297998265709e-09
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.265918312447875e-09
            Iterations: 27
            Function evaluations: 12043
            Gradient evaluations: 27
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3261037733615464e-05
            Iterations: 325
            Function evaluations: 145601
            Gradient evaluations: 325
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.4120143329272583
            Iterations: 82
            Function evaluations: 35018
            Gradient evaluations: 78
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4288426580178096e-09
            Iterations: 27
            Function evaluations: 12097
            Gradient evaluations: 27
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1

In [62]:
#Exportieren nach CSV
portfolio_returns_df_shrinked.to_csv("returnssp500shrinked_neu_2jahre_f.csv")

### Auswertung JSE mit Ledoit Wolf

In [63]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [64]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_JSE_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_JSE_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_JSE_gewichte_min_variance,
    "Maximum-Diversification": shrinked_JSE_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_JSE_gewichte_risk_parity,
    "HRP": shrinked_JSE_gewichte_hrp,
    "HERC": shrinked_JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked_JSE = portfolio_returns_df_shrinked_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.127569592935881e-05
            Iterations: 110
            Function evaluations: 7370
            Gradient evaluations: 110
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.516688127691668
            Iterations: 37
            Function evaluations: 2517
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3221409095318316e-09
            Iterations: 43
            Function evaluations: 2882
            Gradient evaluations: 43
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.17627962303118e-05
            Iterations: 120
            Function evaluations: 8041
            Gradient evaluations: 120
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.594302

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.758582495274235
            Iterations: 34
            Function evaluations: 2449
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.392630882507571e-10
            Iterations: 43
            Function evaluations: 3053
            Gradient evaluations: 43
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.148466486064425e-05
            Iterations: 146
            Function evaluations: 10513
            Gradient evaluations: 146
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.959100698454512
            Iterations: 47
            Function evaluations: 3432
            Gradient evaluations: 47
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.55324108989743e-09
            

Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.686759082642999e-10
            Iterations: 62
            Function evaluations: 4650
            Gradient evaluations: 62
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.660938991320505e-05
            Iterations: 130
            Function evaluations: 9620
            Gradient evaluations: 130
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.954449119581794
            Iterations: 40
            Function evaluations: 2997
            Gradient evaluations: 40
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.273638655061037e-10
            Iterations: 62
            Function evaluations: 4588
            Gradient evaluations: 62
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.751500665

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.130223939197269e-05
            Iterations: 150
            Function evaluations: 11701
            Gradient evaluations: 150
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.166457583824783
            Iterations: 43
            Function evaluations: 3396
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.833221899621061e-10
            Iterations: 73
            Function evaluations: 5694
            Gradient evaluations: 73
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.533910010672287e-05
            Iterations: 161
            Function evaluations: 12719
            Gradient evaluations: 161
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.2306

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.641484633744379
            Iterations: 51
            Function evaluations: 4332
            Gradient evaluations: 51
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.441446692899225e-09
            Iterations: 45
            Function evaluations: 3780
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5029873305379534e-05
            Iterations: 199
            Function evaluations: 16915
            Gradient evaluations: 199
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7245525182697814
            Iterations: 57
            Function evaluations: 4901
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.876168273785637e-09
         

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9219512215761307
            Iterations: 77
            Function evaluations: 10168
            Gradient evaluations: 77
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.553860174373196e-10
            Iterations: 69
            Function evaluations: 9040
            Gradient evaluations: 69
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.9259289568325145e-05
            Iterations: 153
            Function evaluations: 20196
            Gradient evaluations: 153
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9704063205411226
            Iterations: 78
            Function evaluations: 10379
            Gradient evaluations: 78
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.839712038824303e-10
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.482384971240787e-10
            Iterations: 58
            Function evaluations: 8120
            Gradient evaluations: 58
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.584321847174599e-05
            Iterations: 143
            Function evaluations: 20163
            Gradient evaluations: 143
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.1634509008931415
            Iterations: 75
            Function evaluations: 10653
            Gradient evaluations: 75
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.647643080805949e-10
            Iterations: 66
            Function evaluations: 9306
            Gradient evaluations: 66
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.595892

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3789816883335937e-05
            Iterations: 159
            Function evaluations: 24009
            Gradient evaluations: 159
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.250462178659493
            Iterations: 138
            Function evaluations: 20689
            Gradient evaluations: 134
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5321290546151654e-09
            Iterations: 30
            Function evaluations: 4531
            Gradient evaluations: 30
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2980666725844334e-05
            Iterations: 165
            Function evaluations: 24585
            Gradient evaluations: 165
Optimization terminated successfully    (Exit mode 0)
            Current function value: -

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.324394803043635
            Iterations: 98
            Function evaluations: 14506
            Gradient evaluations: 94
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.63729840293553e-09
            Iterations: 33
            Function evaluations: 5049
            Gradient evaluations: 33
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.021628926843486e-05
            Iterations: 174
            Function evaluations: 26623
            Gradient evaluations: 174
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.363285759301371
            Iterations: 99
            Function evaluations: 14670
            Gradient evaluations: 95
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1870917890472644e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.2774597380649484e-09
            Iterations: 43
            Function evaluations: 6622
            Gradient evaluations: 43
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2651698413839362e-05
            Iterations: 185
            Function evaluations: 28490
            Gradient evaluations: 185
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.122905904242547
            Iterations: 106
            Function evaluations: 16334
            Gradient evaluations: 105
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.204625974573796e-09
            Iterations: 42
            Function evaluations: 6468
            Gradient evaluations: 42
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.293

In [65]:
#Exportieren nach CSV
portfolio_returns_df_shrinked_JSE.to_csv("returnsJSEshrinked_neu_2jahre_f.csv")

### Reduzierter S&P500-Bestand

In [66]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [67]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_gewichte_min_variance,
    "Maximum-Diversification": reduziert_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_gewichte_risk_parity,
    "HRP": reduziert_gewichte_hrp,
    "HERC": reduziert_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert = portfolio_returns_df_reduziert.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1696169073000907e-05
            Iterations: 208
            Function evaluations: 21424
            Gradient evaluations: 208
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.490398221986171
            Iterations: 45
            Function evaluations: 4680
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.940029946831117e-09
            Iterations: 21
            Function evaluations: 2163
            Gradient evaluations: 21
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.262584271998927e-05
            Iterations: 219
            Function evaluations: 22558
            Gradient evaluations: 219
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.453

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3866118937362923
            Iterations: 34
            Function evaluations: 3942
            Gradient evaluations: 34
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.274124328133955e-10
            Iterations: 52
            Function evaluations: 5980
            Gradient evaluations: 52
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.3057354978745743e-05
            Iterations: 202
            Function evaluations: 23230
            Gradient evaluations: 202
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3670774984210285
            Iterations: 30
            Function evaluations: 3481
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.297324302459654e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.337178348870864e-10
            Iterations: 79
            Function evaluations: 9639
            Gradient evaluations: 79
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001724443003605545
            Iterations: 140
            Function evaluations: 17080
            Gradient evaluations: 140
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7717011095417352
            Iterations: 24
            Function evaluations: 2941
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.247781693190145e-10
            Iterations: 88
            Function evaluations: 10737
            Gradient evaluations: 88
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000171

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00015119570405768007
            Iterations: 132
            Function evaluations: 16764
            Gradient evaluations: 132
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6357134596813812
            Iterations: 24
            Function evaluations: 3064
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.2259000514896705e-10
            Iterations: 89
            Function evaluations: 11303
            Gradient evaluations: 89
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00014999798285556491
            Iterations: 127
            Function evaluations: 16129
            Gradient evaluations: 127
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0118655213918304
            Iterations: 28
            Function evaluations: 4028
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.032096188979355e-09
            Iterations: 53
            Function evaluations: 7579
            Gradient evaluations: 53
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.4157143326755146e-05
            Iterations: 220
            Function evaluations: 31461
            Gradient evaluations: 220
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.011661636539356
            Iterations: 26
            Function evaluations: 3740
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0204676018925691e-09
        

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.986947756724875
            Iterations: 22
            Function evaluations: 4266
            Gradient evaluations: 22
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.811302744351114e-10
            Iterations: 69
            Function evaluations: 13318
            Gradient evaluations: 69
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011680023346693363
            Iterations: 164
            Function evaluations: 31817
            Gradient evaluations: 164
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9729532788085438
            Iterations: 21
            Function evaluations: 4094
            Gradient evaluations: 21
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.595432080468527e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3703746591617693e-09
            Iterations: 89
            Function evaluations: 17712
            Gradient evaluations: 89
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011031527829904264
            Iterations: 183
            Function evaluations: 36417
            Gradient evaluations: 183
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0699499640263013
            Iterations: 23
            Function evaluations: 4598
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4582346097353091e-09
            Iterations: 89
            Function evaluations: 17712
            Gradient evaluations: 89
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.543645560709816e-05
            Iterations: 258
            Function evaluations: 52633
            Gradient evaluations: 258
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.419168573284899
            Iterations: 36
            Function evaluations: 7381
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.893604414233338e-09
            Iterations: 38
            Function evaluations: 7752
            Gradient evaluations: 38
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.548880015889286e-05
            Iterations: 272
            Function evaluations: 55488
            Gradient evaluations: 272
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3336

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3176605755496587
            Iterations: 36
            Function evaluations: 7415
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.872343901406603e-10
            Iterations: 48
            Function evaluations: 9840
            Gradient evaluations: 48
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.6233177348134735e-05
            Iterations: 273
            Function evaluations: 55965
            Gradient evaluations: 273
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.306815134500901
            Iterations: 31
            Function evaluations: 6386
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.638832788144607e-10
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3259424165940625e-09
            Iterations: 28
            Function evaluations: 5908
            Gradient evaluations: 28
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.8161469221119367e-05
            Iterations: 265
            Function evaluations: 55915
            Gradient evaluations: 265
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1074149829671565
            Iterations: 58
            Function evaluations: 12294
            Gradient evaluations: 58
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.8669779259988844e-09
            Iterations: 28
            Function evaluations: 5908
            Gradient evaluations: 28
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.650

In [68]:
#Exportieren nach CSV
portfolio_returns_df_reduziert.to_csv("returnsreduziert_neu_2jahre_f.csv")

### Reduziert und Ledoit-Shrinkage

In [69]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [70]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 24
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 126
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_shrinked_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_shrinked_gewichte_min_variance,
    "Maximum-Diversification": reduziert_shrinked_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_shrinked_gewichte_risk_parity,
    "HRP": reduziert_shrinked_gewichte_hrp,
    "HERC": reduziert_shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert_shrinked = portfolio_returns_df_reduziert_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1292669267537483e-05
            Iterations: 208
            Function evaluations: 21424
            Gradient evaluations: 208
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5307893769663505
            Iterations: 49
            Function evaluations: 5094
            Gradient evaluations: 49
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.729483386721757e-09
            Iterations: 21
            Function evaluations: 2163
            Gradient evaluations: 21
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.226814812522563e-05
            Iterations: 212
            Function evaluations: 21836
            Gradient evaluations: 212
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.49

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.413696671523841
            Iterations: 32
            Function evaluations: 3713
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.756152634792623e-10
            Iterations: 52
            Function evaluations: 5981
            Gradient evaluations: 52
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.263883641483559e-05
            Iterations: 198
            Function evaluations: 22770
            Gradient evaluations: 198
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3928242257456014
            Iterations: 32
            Function evaluations: 3712
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7647027634227055e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1303876925739054e-10
            Iterations: 89
            Function evaluations: 10858
            Gradient evaluations: 89
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00017219052103742102
            Iterations: 109
            Function evaluations: 13298
            Gradient evaluations: 109
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7944576641374466
            Iterations: 23
            Function evaluations: 2822
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1402355345724704e-10
            Iterations: 89
            Function evaluations: 10858
            Gradient evaluations: 89
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00015193857339267964
            Iterations: 92
            Function evaluations: 11684
            Gradient evaluations: 92
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.667285330936374
            Iterations: 22
            Function evaluations: 2810
            Gradient evaluations: 22
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.081026870326128e-10
            Iterations: 89
            Function evaluations: 11303
            Gradient evaluations: 89
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00015072008265559354
            Iterations: 111
            Function evaluations: 14097
            Gradient evaluations: 111
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.671

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0281876515682655
            Iterations: 28
            Function evaluations: 4030
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0059912039871809e-09
            Iterations: 53
            Function evaluations: 7579
            Gradient evaluations: 53
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.397047934426979e-05
            Iterations: 209
            Function evaluations: 29887
            Gradient evaluations: 209
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.02810918978827
            Iterations: 29
            Function evaluations: 4173
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.952862034734784e-10
          

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.025787916583871
            Iterations: 25
            Function evaluations: 4846
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.238644105657307e-10
            Iterations: 70
            Function evaluations: 13510
            Gradient evaluations: 70
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011359538942206801
            Iterations: 156
            Function evaluations: 30264
            Gradient evaluations: 156
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0117829910945644
            Iterations: 24
            Function evaluations: 4678
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.976322930812202e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2664773178264587e-09
            Iterations: 90
            Function evaluations: 17910
            Gradient evaluations: 90
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00010777626009430061
            Iterations: 171
            Function evaluations: 34029
            Gradient evaluations: 171
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.104034634272241
            Iterations: 24
            Function evaluations: 4798
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3476144782946842e-09
            Iterations: 90
            Function evaluations: 17910
            Gradient evaluations: 90
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.441975496302284e-05
            Iterations: 266
            Function evaluations: 54264
            Gradient evaluations: 266
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.443376310374908
            Iterations: 37
            Function evaluations: 7585
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8192914334073768e-09
            Iterations: 38
            Function evaluations: 7753
            Gradient evaluations: 38
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.476195659704855e-05
            Iterations: 258
            Function evaluations: 52633
            Gradient evaluations: 258
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.351

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3344344660205065
            Iterations: 37
            Function evaluations: 7622
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.0553487519494934e-09
            Iterations: 37
            Function evaluations: 7586
            Gradient evaluations: 37
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.549293646683711e-05
            Iterations: 258
            Function evaluations: 52890
            Gradient evaluations: 258
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3242132434895395
            Iterations: 31
            Function evaluations: 6384
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.394000061962707e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.169942631632527e-09
            Iterations: 29
            Function evaluations: 6119
            Gradient evaluations: 29
2022-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.734441498693656e-05
            Iterations: 265
            Function evaluations: 55915
            Gradient evaluations: 265
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1425165041410796
            Iterations: 60
            Function evaluations: 12715
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.7209933920179515e-09
            Iterations: 28
            Function evaluations: 5908
            Gradient evaluations: 28
2023-01-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.57660

In [71]:
#Exportieren nach CSV
portfolio_returns_df_reduziert_shrinked.to_csv("returnsreduziertshrinked_neu_2jahre_f.csv")